# Data Manipulation

## Primary Dataset

### Menstrual Cycle Regularity

Menstrual cycle regularity in the past 12 months was assessed by asking participants whether they experienced regular periods. Participants were classified as having regular cycles if they reported consistent menstrual bleeding patterns. Participants who reported irregular cycles were classified as having irregular cycles, provided that no notable medical or physiological reasons were present.

**Participants were excluded from the irregularity classification if they had conditions or circumstances that naturally prevent menstruation, including: pregnancy, postpartum or breastfeeding-related amenorrhea, hysterectomy, or menopause.** This approach ensures that the measure of cycle irregularity reflects only those individuals for whom menstrual cycles would be expected under normal conditions.

In [1]:
#Imports
import pandas as pd
import numpy as np

#### Reproductive Health

In [2]:
# Load reproductive health dataset XPT file
cycle_17_to_pre20 = pd.read_sas('data/P_RHQ.XPT')
cycle_21_to_23 = pd.read_sas('data/RHQ_L.XPT')

# See what’s inside
print(cycle_17_to_pre20.shape)
print(cycle_21_to_23.shape)

(5314, 32)
(3917, 13)


In [3]:
# Find overlapping columns
cycle_common_cols = cycle_17_to_pre20.columns.intersection(cycle_21_to_23.columns)

# Keep only overlapping columns
cycle_df = pd.concat([cycle_17_to_pre20[cycle_common_cols], cycle_21_to_23[cycle_common_cols]], axis=0, ignore_index=True)

In [4]:
# Rename variables of interest
cycle_df.rename(columns={
    'RHQ031': 'Regular_Periods',
    'RHD043': 'Irregular_Period_Reason'
}, inplace=True)

In [5]:
# Map categorical values (others become NaN automatically)
cycle_df['Regular_Periods'] = cycle_df['Regular_Periods'].map({
    1: 'Yes', 
    2: 'No'
})

In [6]:
#Drop rows where Regular_Periods = NaN
cycle_df.dropna(subset=['Regular_Periods'],inplace=True)

# Drop rows where Irregular_Period_Reason = pregnancy (1), breast feeding (2), hysterectomy (3), or menopause (7)
cycle_df = cycle_df[~cycle_df['Irregular_Period_Reason'].isin([1, 2, 3, 7])]

In [7]:
#Retain variables of interest only
cycle_df = cycle_df[['SEQN', 'Regular_Periods']]

In [8]:
cycle_df

,SEQN,Regular_Periods
0,109264.0,Yes
1,109266.0,Yes
2,109277.0,Yes
3,109279.0,Yes
4,109284.0,Yes
...,...,...
9214,142263.0,Yes
9217,142269.0,No
9220,142280.0,Yes
9221,142283.0,Yes


In [9]:
#Confirm only specified values remain in fields
print(cycle_df['Regular_Periods'].value_counts())

#Check value & N/A counts
print('Values: ',cycle_df['Regular_Periods'].value_counts().sum())
print('N/A: ',cycle_df['Regular_Periods'].isna().sum())
print('Total: ',cycle_df['Regular_Periods'].isna().value_counts().sum())

Regular_Periods
Yes    4063
No      396
Name: count, dtype: int64
Values:  4459
N/A:  0
Total:  4459


#### Demographics

In [10]:
# Load demographic dataset XPT file
demo_17_to_pre20 = pd.read_sas('data/P_DEMO.XPT')
demo_21_to_23 = pd.read_sas('data/DEMO_L.XPT')

# See what’s inside
print(demo_17_to_pre20.shape)
print(demo_21_to_23.shape)

(15560, 29)
(11933, 27)


In [11]:
# Find overlapping columns
demo_common_cols = demo_17_to_pre20.columns.intersection(demo_21_to_23.columns)

# Keep only overlapping columns
demo_df = pd.concat([demo_17_to_pre20[demo_common_cols], demo_21_to_23[demo_common_cols]], axis=0, ignore_index=True)

In [12]:
# Merge cycle_df and demo_df on SEQN (common ID)
demo_df = pd.merge(cycle_df, demo_df, on='SEQN', how='left')

In [13]:
# Rename variables of interest
#Already filtering by sex based on cycle data only coming from females
demo_df.rename(columns={
    'RIDRETH3': 'Race',
    'RIDAGEYR': 'Age',
    'DMDEDUC2': 'Education',
    'INDFMPIR': 'Family_Income_to_Poverty_Ratio'
}, inplace=True)

In [14]:
# Map categorical values
demo_df['Race'] = demo_df['Race'].map({
    1: 'Mexican American', 
    2: 'Other Hispanic',
    3: 'Non-Hispanic White', 
    4: 'Non-Hispanic Black',
    6: 'Non-Hispanic Asian',
    7: 'Other Race - Including Multi-Racial'
})
demo_df['Education'] = demo_df['Education'].map({
    1: 'Less than 9th grade',
    2: '9-11th grade',
    3: 'High school/GED',
    4: 'Some college/AA',
    5: 'College graduate or above'
})

In [15]:
#Drop rows where Age <18 & >50
#18 lower limit based on some datasets only containing data from adults (18+)
#50 upper limit based on medical research: https://www.mayoclinic.org/diseases-conditions/menopause/symptoms-causes/syc-20353397
demo_df = demo_df[(demo_df['Age'] >= 18) & (demo_df['Age'] <= 50)]

In [16]:
#Retain variables of interest only
demo_df = demo_df[['SEQN', 'Regular_Periods', 'Race', 'Age', 'Education', 'Family_Income_to_Poverty_Ratio']]

In [17]:
demo_df

,SEQN,Regular_Periods,Race,Age,Education,Family_Income_to_Poverty_Ratio
1,109266.0,Yes,Non-Hispanic Asian,29.0,College graduate or above,5.00
4,109284.0,Yes,Mexican American,44.0,9-11th grade,NaN
5,109291.0,Yes,Non-Hispanic Asian,42.0,College graduate or above,NaN
7,109297.0,Yes,Non-Hispanic Asian,30.0,Some college/AA,NaN
8,109312.0,Yes,Non-Hispanic White,48.0,College graduate or above,5.00
...,...,...,...,...,...,...
4454,142263.0,Yes,Non-Hispanic White,44.0,College graduate or above,NaN
4455,142269.0,No,Non-Hispanic Black,32.0,Some college/AA,0.74
4456,142280.0,Yes,Other Race - Including Multi-Racial,23.0,Some college/AA,1.40
4457,142283.0,Yes,Other Race - Including Multi-Racial,29.0,High school/GED,1.04


In [18]:
#Confirm only specified values remain in fields
print(demo_df['Age'].value_counts())

#Check value & N/A counts
print('Values: ',demo_df['Age'].value_counts().sum())
print('N/A: ',demo_df['Age'].isna().sum())
print('Total: ',demo_df['Age'].isna().value_counts().sum())

Age
19.0    163
18.0    141
39.0    124
34.0    114
29.0    110
31.0    107
33.0    104
37.0    104
32.0    101
41.0    100
23.0    100
28.0     97
44.0     96
43.0     96
35.0     96
25.0     96
30.0     95
42.0     94
27.0     94
38.0     92
40.0     92
36.0     91
21.0     89
24.0     87
22.0     85
45.0     83
20.0     81
47.0     81
26.0     78
48.0     74
46.0     72
50.0     56
49.0     50
Name: count, dtype: int64
Values:  3143
N/A:  0
Total:  3143


In [19]:
#Filter cycle_df by age 18-50 now that we have age range from demo_df
cycle_df = demo_df[['SEQN', 'Regular_Periods']]

In [20]:
cycle_df

,SEQN,Regular_Periods
1,109266.0,Yes
4,109284.0,Yes
5,109291.0,Yes
7,109297.0,Yes
8,109312.0,Yes
...,...,...
4454,142263.0,Yes
4455,142269.0,No
4456,142280.0,Yes
4457,142283.0,Yes


## Secondary Dataset
- Body Measures (height, weight, BMI)
- Smoking (smoking status, frequency, history)
- Alcohol Consumption
- Physical Activity (activity type, frequency, duration, intensity)
- Demographics (age, race/ethnicity, education, income)
- Dietary Data (diet quality, caloric and nutrient intake)
- Sleep Disorders (sleep quality, trouble sleeping, sleep duration)

#### Body Measures (height, weight, BMI)

In [21]:
# Load body measures dataset XPT file
bmi_17_to_pre20 = pd.read_sas('data/P_BMX.XPT')
bmi_21_to_23 = pd.read_sas('data/BMX_L.XPT')

weight_17_to_pre20 = pd.read_sas('data/P_WHQ.XPT')
weight_21_to_23 = pd.read_sas('data/WHQ_L.XPT')

# See what’s inside
print(bmi_17_to_pre20.shape)
print(bmi_21_to_23.shape)

print(weight_17_to_pre20.shape)
print(weight_21_to_23.shape)

(14300, 22)
(8860, 22)
(10195, 35)
(8501, 5)


In [22]:
# Find overlapping columns
bmi_common_cols = bmi_17_to_pre20.columns.intersection(bmi_21_to_23.columns)
weight_common_cols = weight_17_to_pre20.columns.intersection(weight_21_to_23.columns)

# Keep only overlapping columns
bmi_df = pd.concat([bmi_17_to_pre20[bmi_common_cols], bmi_21_to_23[bmi_common_cols]], axis=0, ignore_index=True)
weight_df = pd.concat([weight_17_to_pre20[weight_common_cols], weight_21_to_23[weight_common_cols]], axis=0, ignore_index=True)

In [23]:
# Merge cycle_df and bmi_df on SEQN (common ID)
bmi_df = pd.merge(cycle_df, bmi_df, on='SEQN', how='left')
bmi_df = pd.merge(bmi_df, weight_df, on='SEQN', how='left')

In [24]:
# Rename variables of interest
bmi_df.rename(columns={
    'BMXBMI': 'BMI',
    'BMXWAIST': 'Waist_circum_cm',
    'BMXHIP': 'Hip_Circum_cm',
    'WHQ070': 'Attempted_Weight_Loss',
    'WHD010': 'Height_in',
    'WHD020': 'Weight_lbs',
    'WHD050': 'Prev_Weight_lbs'
}, inplace=True)

In [25]:
# Map categorical values
bmi_df['Attempted_Weight_Loss'] = bmi_df['Attempted_Weight_Loss'].map({
    1: 'Yes',
    2: 'No'
})
#Using replace over map since range values need to be retained
bmi_df['Height_in'] = bmi_df['Height_in'].replace(7777, np.nan).replace(9999, np.nan)
bmi_df['Weight_lbs'] = bmi_df['Weight_lbs'].replace(7777, np.nan).replace(9999, np.nan)
bmi_df['Prev_Weight_lbs'] = bmi_df['Prev_Weight_lbs'].replace(7777, np.nan).replace(9999, np.nan)

In [26]:
# No additional rows need to be dropped

In [27]:
# Add calculation column for weight loss
bmi_df['Weight_Change_lbs'] = bmi_df['Prev_Weight_lbs'] - bmi_df['Weight_lbs']

In [28]:
#Retain variables of interest only
bmi_df = bmi_df[['SEQN','BMI', 'Waist_circum_cm', 'Hip_Circum_cm', 'Attempted_Weight_Loss', 'Height_in', 'Weight_lbs', 'Weight_Change_lbs']]

In [29]:
bmi_df

,SEQN,BMI,Waist_circum_cm,Hip_Circum_cm,Attempted_Weight_Loss,Height_in,Weight_lbs,Weight_Change_lbs
0,109266.0,37.8,117.9,126.1,Yes,64.0,210.0,-10.0
1,109284.0,39.1,103.1,125.5,Yes,60.0,178.0,-8.0
2,109291.0,31.3,NaN,NaN,No,64.0,148.0,-8.0
3,109297.0,23.2,73.2,92.5,Yes,60.0,120.0,-5.0
4,109312.0,23.9,95.7,96.3,No,66.0,139.0,0.0
...,...,...,...,...,...,...,...,...
3138,142263.0,22.6,80.4,95.0,No,65.0,135.0,0.0
3139,142269.0,NaN,149.3,147.2,No,64.0,350.0,0.0
3140,142280.0,38.4,124.1,130.0,Yes,69.0,250.0,20.0
3141,142283.0,45.8,137.2,145.7,Yes,67.0,275.0,5.0


In [30]:
#Confirm only specified values remain in fields
print(bmi_df['BMI'].value_counts())

#Check value & N/A counts
print('Values: ',bmi_df['BMI'].value_counts().sum())
print('N/A: ',bmi_df['BMI'].isna().sum())
print('Total: ',bmi_df['BMI'].isna().value_counts().sum())

BMI
25.2    24
23.5    24
26.1    24
21.6    23
22.7    22
        ..
49.0     1
54.7     1
64.2     1
14.8     1
45.1     1
Name: count, Length: 407, dtype: int64
Values:  3112
N/A:  31
Total:  3143


#### Smoking (smoking status, frequency, history)

In [31]:
# Load smoking dataset XPT files
smoking_cig_17_to_pre20 = pd.read_sas('data/P_SMQ.XPT')
smoking_cig_21_to_23 = pd.read_sas('data/SMQ_L.XPT')

smoking_house_17_to_pre20 = pd.read_sas('data/P_SMQFAM.XPT')
smoking_house_21_to_23 = pd.read_sas('data/SMQFAM_L.XPT')

smoking_tobacco_17_to_pre20 = pd.read_sas('data/P_SMQRTU.XPT')
smoking_tobacco_21_to_23 = pd.read_sas('data/SMQRTU_L.XPT')

# See what’s inside
print(smoking_cig_17_to_pre20.shape)
print(smoking_cig_21_to_23.shape)

print(smoking_house_17_to_pre20.shape)
print(smoking_house_21_to_23.shape)

print(smoking_tobacco_17_to_pre20.shape)
print(smoking_tobacco_21_to_23.shape)

(11137, 16)
(9015, 9)
(15560, 3)
(11933, 3)
(10409, 25)
(7199, 21)


In [32]:
# Find overlapping columns
smoking_cig_common_cols = smoking_cig_17_to_pre20.columns.intersection(smoking_cig_21_to_23.columns)
smoking_house_common_cols = smoking_house_17_to_pre20.columns.intersection(smoking_house_21_to_23.columns)
smoking_tobacco_common_cols = smoking_tobacco_17_to_pre20.columns.intersection(smoking_tobacco_21_to_23.columns)

# Keep only overlapping columns
smoking_cig = pd.concat([smoking_cig_17_to_pre20[smoking_cig_common_cols], smoking_cig_21_to_23[smoking_cig_common_cols]], axis=0, ignore_index=True)
smoking_house = pd.concat([smoking_house_17_to_pre20[smoking_house_common_cols], smoking_house_21_to_23[smoking_house_common_cols]], axis=0, ignore_index=True)
smoking_tobacco = pd.concat([smoking_tobacco_17_to_pre20[smoking_tobacco_common_cols], smoking_tobacco_21_to_23[smoking_tobacco_common_cols]], axis=0, ignore_index=True)

In [33]:
# Merge cycle_df and smoking dfs on SEQN (common ID)
smoking_df = pd.merge(cycle_df, smoking_cig, on='SEQN', how='left')
smoking_df = pd.merge(smoking_df, smoking_house, on='SEQN', how='left')
smoking_df = pd.merge(smoking_df, smoking_tobacco, on='SEQN', how='left')

In [34]:
# Rename variables of interest
smoking_df.rename(columns={
    'SMQ020': 'Smoked_100', 
    'SMQ040': 'Active_Smoker', 
    'SMQ621': 'Lifetime_Cig_Count',
    'SMD460': 'Household_Smoker_Count',
    'SMQ725': 'Last_Cig',
}, inplace=True)

In [35]:
# Map categorical values
smoking_df['Smoked_100'] = smoking_df['Smoked_100'].map({
    1: 'Yes', 
    2: 'No'})
smoking_df['Active_Smoker'] = smoking_df['Active_Smoker'].map({
    1: 'Every day', 
    2: 'Some days',
    3: 'Not at all'})
smoking_df['Lifetime_Cig_Count'] = smoking_df['Lifetime_Cig_Count'].map({
    1: 0, 
    2: '<1',
    3: 1,
    4: '2 to 5',
    5: '6 to 15',
    6: '16 to 25',
    7: '26 to 99',
    8: '100+'})
smoking_df['Household_Smoker_Count'] = smoking_df['Household_Smoker_Count'].map({
    5.397605346934028e-79: 0, #0 is being interpreted as 5.397605346934028e-79, the lowest IBM float value
    1: 1,
    2: '2+'})
smoking_df['Last_Cig'] = smoking_df['Last_Cig'].map({
    1: 'Today', 
    2: 'Yesterday',
    3: '3 to 5 days ago'})

In [36]:
# No additional rows need to be dropped

In [37]:
#Retain variables of interest only
smoking_df = smoking_df[['SEQN', 'Regular_Periods', 'Smoked_100', 'Active_Smoker', 'Lifetime_Cig_Count', 'Household_Smoker_Count', 'Last_Cig']]

In [38]:
smoking_df

,SEQN,Regular_Periods,Smoked_100,Active_Smoker,Lifetime_Cig_Count,Household_Smoker_Count,Last_Cig
0,109266.0,Yes,No,NaN,NaN,0,NaN
1,109284.0,Yes,No,NaN,NaN,0,NaN
2,109291.0,Yes,No,NaN,NaN,0,NaN
3,109297.0,Yes,No,NaN,NaN,0,Yesterday
4,109312.0,Yes,No,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...
3138,142263.0,Yes,No,NaN,NaN,0,NaN
3139,142269.0,No,No,NaN,NaN,0,NaN
3140,142280.0,Yes,No,NaN,NaN,0,NaN
3141,142283.0,Yes,No,NaN,NaN,0,NaN


In [39]:
#Confirm only specified values remain in fields
print(smoking_df['Smoked_100'].value_counts())

#Check value & N/A counts
print('Values: ',smoking_df['Smoked_100'].value_counts().sum())
print('N/A: ',smoking_df['Smoked_100'].isna().sum())
print('Total: ',smoking_df['Smoked_100'].isna().value_counts().sum())

Smoked_100
No     2315
Yes     825
Name: count, dtype: int64
Values:  3140
N/A:  3
Total:  3143


#### Alcohol

In [40]:
# Load alcohol dataset XPT files
alcohol_17_to_pre20 = pd.read_sas('data/P_ALQ.XPT')
alcohol_21_to_23 = pd.read_sas('data/ALQ_L.XPT')

# See what’s inside
print(alcohol_17_to_pre20.shape)
print(alcohol_21_to_23.shape)

(8965, 10)
(6337, 9)


In [41]:
# Find overlapping columns
alcohol_common_cols = alcohol_17_to_pre20.columns.intersection(alcohol_21_to_23.columns)

# Keep only overlapping columns
alcohol_df = pd.concat([alcohol_17_to_pre20[alcohol_common_cols], alcohol_21_to_23[alcohol_common_cols]], axis=0, ignore_index=True)

In [42]:
# Merge cycle_df and alcohol_df on SEQN (common ID)
alcohol_df = pd.merge(cycle_df, alcohol_df, on='SEQN', how='left')

In [43]:
# Rename variables of interest
alcohol_df.rename(columns={
    'ALQ121': 'Alc_Freq', 
    'ALQ130': 'Daily_Drinks',
}, inplace=True)

In [44]:
# Map categorical values
alcohol_df['Alc_Freq'] = alcohol_df['Alc_Freq'].map({
    5.397605346934028e-79: 'Never', #0 is being interpreted as 5.397605346934028e-79, the lowest IBM float value
    1: 'Every Day',
    2: 'Nearly Every Day',
    3: '3-4x/wk',
    4: '2x/wk',
    5: '1x/wk',
    6: '2-3x/mo',
    7: '1x/mo',
    8: '7-11x/yr',
    9: '3-6x/yr',
    10: '1-2x/yr'
})
#Using replace over map since range values need to be retained
alcohol_df['Daily_Drinks'] = alcohol_df['Daily_Drinks'].replace(15, '15+' ).replace(777, np.nan).replace(999, np.nan)

In [45]:
# No additional rows need to be dropped

In [46]:
#Retain variables of interest only
alcohol_df = alcohol_df[['SEQN', 'Alc_Freq', 'Daily_Drinks']]

In [47]:
alcohol_df

,SEQN,Alc_Freq,Daily_Drinks
0,109266.0,1-2x/yr,1.0
1,109284.0,NaN,NaN
2,109291.0,NaN,NaN
3,109297.0,2-3x/mo,2.0
4,109312.0,3-6x/yr,1.0
...,...,...,...
3138,142263.0,Never,NaN
3139,142269.0,3-6x/yr,2.0
3140,142280.0,1-2x/yr,3.0
3141,142283.0,2-3x/mo,2.0


In [48]:
#Confirm only specified values remain in fields
print(alcohol_df['Alc_Freq'].value_counts())

#Check value & N/A counts
print('Values: ',alcohol_df['Alc_Freq'].value_counts().sum())
print('N/A: ',alcohol_df['Alc_Freq'].isna().sum())
print('Total: ',alcohol_df['Alc_Freq'].isna().value_counts().sum())

Alc_Freq
2-3x/mo             510
3-6x/yr             401
1-2x/yr             391
Never               289
1x/mo               250
2x/wk               239
1x/wk               231
7-11x/yr            194
3-4x/wk             171
Nearly Every Day     64
Every Day            30
Name: count, dtype: int64
Values:  2770
N/A:  373
Total:  3143


#### Physical Activity

In [49]:
# Load physical activity dataset XPT files
exercise_17_to_pre20 = pd.read_sas('data/P_PAQ.XPT')
exercise_21_to_23 = pd.read_sas('data/PAQ_L.XPT')

# See what’s inside
print(exercise_17_to_pre20.shape)
print(exercise_21_to_23.shape)

(9693, 17)
(8153, 8)


In [50]:
# Keep all columns
exercise_df = pd.concat([exercise_17_to_pre20, exercise_21_to_23], axis=0, ignore_index=True)

In [51]:
# Merge cycle_df and exercise_df on SEQN (common ID)
exercise_df = pd.merge(cycle_df, exercise_df, on='SEQN', how='left')

In [52]:
# Rename variables of interest
exercise_df.rename(columns={
    'PAD680': 'Sedentary_Mins',
    'PAD790Q': 'Moderate_LTPA_Freq_1',
    'PAD790U': 'Moderate_LTPA_Units',
    'PAD800': 'Moderate_LTPA_Mins_1',
    'PAD810Q': 'Vigorous_LTPA_Freq_1',
    'PAD810U': 'Vigorous_LTPA_Units',
    'PAD820': 'Vigorous_LTPA_Mins_1',
    'PAQ610': 'Vigorous_LTPA_Freq_Work',
    'PAD615': 'Vigorous_LTPA_Mins_Work',
    'PAQ625': 'Moderate_LTPA_Freq_Work',
    'PAD630': 'Moderate_LTPA_Mins_Work',
    'PAQ655': 'Vigorous_LTPA_Freq_Rec',
    'PAD660': 'Vigorous_LTPA_Mins_Rec',
    'PAQ670': 'Moderate_LTPA_Freq_Rec',
    'PAD675': 'Moderate_LTPA_Mins_Rec'
}, inplace=True)

In [53]:
# Map categorical values
exercise_df['Moderate_LTPA_Units_Math'] = exercise_df['Moderate_LTPA_Units'].map({
    b'D': 365,
    b'M': 12,
    b'W': 52,
    b'Y': 1
})
exercise_df['Moderate_LTPA_Units_Text'] = exercise_df['Moderate_LTPA_Units'].map({
    b'D': 'day(s)',
    b'M': 'month(s)',
    b'W': 'week(s)',
    b'Y': 'year(s)'
})
exercise_df['Vigorous_LTPA_Units_Math'] = exercise_df['Vigorous_LTPA_Units'].map({
    b'D': 365,
    b'M': 12,
    b'W': 52,
    b'Y': 1
})
exercise_df['Vigorous_LTPA_Units_Text'] = exercise_df['Vigorous_LTPA_Units'].map({
    b'D': 'day(s)',
    b'M': 'month(s)',
    b'W': 'week(s)',
    b'Y': 'year(s)'
})

# Using replace over map since range values need to be retained
exercise_df['Sedentary_Mins'] = exercise_df['Sedentary_Mins'].replace(7777, np.nan).replace(9999, np.nan)

exercise_df['Moderate_LTPA_Freq_1'] = exercise_df['Moderate_LTPA_Freq_1'].replace(5.397605346934028e-79, 0).replace(7777, np.nan).replace(9999, np.nan)
exercise_df['Moderate_LTPA_Mins_1'] = exercise_df['Moderate_LTPA_Mins_1'].replace(7777, np.nan).replace(9999, np.nan)

exercise_df['Vigorous_LTPA_Freq_1'] = exercise_df['Vigorous_LTPA_Freq_1'].replace(5.397605346934028e-79, 0).replace(7777, np.nan).replace(9999, np.nan)
exercise_df['Vigorous_LTPA_Mins_1'] = exercise_df['Vigorous_LTPA_Mins_1'].replace(7777, np.nan).replace(9999, np.nan)

exercise_df['Moderate_LTPA_Freq_Work'] = exercise_df['Moderate_LTPA_Freq_Work'].replace(77, np.nan).replace(99, np.nan)
exercise_df['Moderate_LTPA_Mins_Work'] = exercise_df['Moderate_LTPA_Mins_Work'].replace(7777, np.nan).replace(9999, np.nan)

exercise_df['Vigorous_LTPA_Freq_Work'] = exercise_df['Vigorous_LTPA_Freq_Work'].replace(77, np.nan).replace(99, np.nan)
exercise_df['Vigorous_LTPA_Mins_Work'] = exercise_df['Vigorous_LTPA_Mins_Work'].replace(7777, np.nan).replace(9999, np.nan)

exercise_df['Moderate_LTPA_Freq_Rec'] = exercise_df['Moderate_LTPA_Freq_Rec'].replace(77, np.nan).replace(99, np.nan)
exercise_df['Moderate_LTPA_Mins_Rec'] = exercise_df['Moderate_LTPA_Mins_Rec'].replace(7777, np.nan).replace(9999, np.nan)

exercise_df['Vigorous_LTPA_Freq_Rec'] = exercise_df['Vigorous_LTPA_Freq_Rec'].replace(77, np.nan).replace(99, np.nan)
exercise_df['Vigorous_LTPA_Mins_Rec'] = exercise_df['Vigorous_LTPA_Mins_Rec'].replace(7777, np.nan).replace(9999, np.nan)

In [54]:
# No additional rows need to be dropped

In [55]:
# Add calculation columns for hrs per day & year
exercise_df['Sedentary_Hrs_Wk'] = exercise_df['Sedentary_Mins'] / 60 * 7

# Add calculation column for hrs per week avg
exercise_df['Moderate_LTPA_Hrs_Wk_1'] = exercise_df['Moderate_LTPA_Freq_1'] * exercise_df['Moderate_LTPA_Units_Math'].fillna(0).astype(int) * exercise_df['Moderate_LTPA_Mins_1'] / 60 / 52
exercise_df['Vigorous_LTPA_Hrs_Wk_1'] = exercise_df['Vigorous_LTPA_Freq_1'] * exercise_df['Vigorous_LTPA_Units_Math'].fillna(0).astype(int) * exercise_df['Vigorous_LTPA_Mins_1'] / 60 / 52
exercise_df['Moderate_LTPA_Hrs_Wk_Work'] = exercise_df['Moderate_LTPA_Freq_Work'] * exercise_df['Moderate_LTPA_Mins_Work'] / 60
exercise_df['Moderate_LTPA_Hrs_Wk_Rec'] = exercise_df['Moderate_LTPA_Freq_Rec'] * exercise_df['Moderate_LTPA_Mins_Rec'] / 60
exercise_df['Vigorous_LTPA_Hrs_Wk_Work'] = exercise_df['Vigorous_LTPA_Freq_Work'] * exercise_df['Vigorous_LTPA_Mins_Work'] / 60
exercise_df['Vigorous_LTPA_Hrs_Wk_Rec'] = exercise_df['Vigorous_LTPA_Freq_Rec'] * exercise_df['Vigorous_LTPA_Mins_Rec'] / 60

# Combine work & rec hrs per week fields
exercise_df['Moderate_LTPA_Hrs_Wk_2'] = exercise_df['Moderate_LTPA_Hrs_Wk_Work'] + exercise_df['Moderate_LTPA_Hrs_Wk_Rec']
exercise_df['Vigorous_LTPA_Hrs_Wk_2'] = exercise_df['Vigorous_LTPA_Hrs_Wk_Work'] + exercise_df['Vigorous_LTPA_Hrs_Wk_Rec']

# Combine hrs per week fields from 17-20 & 21-23 files
exercise_df['Moderate_LTPA_Hrs_Wk'] = exercise_df['Moderate_LTPA_Hrs_Wk_1'].combine_first(exercise_df['Moderate_LTPA_Hrs_Wk_2'])
exercise_df['Vigorous_LTPA_Hrs_Wk'] = exercise_df['Vigorous_LTPA_Hrs_Wk_1'].combine_first(exercise_df['Vigorous_LTPA_Hrs_Wk_2'])

# Combine work & rec freq fields
exercise_df['Moderate_LTPA_Freq_2'] = exercise_df['Moderate_LTPA_Freq_Work'] + exercise_df['Moderate_LTPA_Freq_Rec']
exercise_df['Vigorous_LTPA_Freq_2'] = exercise_df['Vigorous_LTPA_Freq_Work'] + exercise_df['Vigorous_LTPA_Freq_Rec']

# Combine work & rec mins fields
exercise_df['Moderate_LTPA_Mins_2'] = exercise_df['Moderate_LTPA_Mins_Work'] + exercise_df['Moderate_LTPA_Mins_Rec']
exercise_df['Vigorous_LTPA_Mins_2'] = exercise_df['Vigorous_LTPA_Mins_Work'] + exercise_df['Vigorous_LTPA_Mins_Rec']

# Modify freq field to include units
exercise_df['Moderate_LTPA_Freq_Units_1'] = exercise_df['Moderate_LTPA_Freq_1'].astype(str) + 'x/ ' + exercise_df['Moderate_LTPA_Units_Text']
exercise_df['Vigorous_LTPA_Freq_Units_1'] = exercise_df['Vigorous_LTPA_Freq_1'].astype(str) + 'x/ ' + exercise_df['Vigorous_LTPA_Units_Text']
exercise_df['Moderate_LTPA_Freq_Units_2'] = exercise_df['Moderate_LTPA_Freq_2'].where(exercise_df['Moderate_LTPA_Freq_2'].isna(), exercise_df['Moderate_LTPA_Freq_2'].astype(str) + 'x/ week')
exercise_df['Vigorous_LTPA_Freq_Units_2'] = exercise_df['Vigorous_LTPA_Freq_2'].where(exercise_df['Vigorous_LTPA_Freq_2'].isna(), exercise_df['Vigorous_LTPA_Freq_2'].astype(str) + 'x/ week')

# Combine freq fields from 17-20 & 21-23 files
exercise_df['Moderate_LTPA_Freq'] = exercise_df['Moderate_LTPA_Freq_Units_1'].combine_first(exercise_df['Moderate_LTPA_Freq_Units_2'])
exercise_df['Vigorous_LTPA_Freq'] = exercise_df['Vigorous_LTPA_Freq_Units_1'].combine_first(exercise_df['Vigorous_LTPA_Freq_Units_2'])

# Combine mins fields from 17-20 & 21-23 files
exercise_df['Moderate_LTPA_Mins'] = exercise_df['Moderate_LTPA_Mins_1'].combine_first(exercise_df['Moderate_LTPA_Mins_2'])
exercise_df['Vigorous_LTPA_Mins'] = exercise_df['Vigorous_LTPA_Mins_1'].combine_first(exercise_df['Vigorous_LTPA_Mins_2'])

In [56]:
#Retain variables of interest only
exercise_df = exercise_df[['SEQN', 'Regular_Periods', 'Sedentary_Hrs_Wk', 'Moderate_LTPA_Freq', 'Moderate_LTPA_Mins', 'Moderate_LTPA_Hrs_Wk', 'Vigorous_LTPA_Freq', 'Vigorous_LTPA_Mins', 'Vigorous_LTPA_Hrs_Wk']]

In [57]:
exercise_df

,SEQN,Regular_Periods,Sedentary_Hrs_Wk,Moderate_LTPA_Freq,Moderate_LTPA_Mins,Moderate_LTPA_Hrs_Wk,Vigorous_LTPA_Freq,Vigorous_LTPA_Mins,Vigorous_LTPA_Hrs_Wk
0,109266.0,Yes,56.0,NaN,NaN,NaN,NaN,NaN,NaN
1,109284.0,Yes,42.0,NaN,NaN,NaN,NaN,NaN,NaN
2,109291.0,Yes,63.0,NaN,NaN,NaN,NaN,NaN,NaN
3,109297.0,Yes,70.0,NaN,NaN,NaN,NaN,NaN,NaN
4,109312.0,Yes,21.0,12.0x/ week,480.0,52.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3138,142263.0,Yes,56.0,5.0x/ week(s),90.0,7.5,5.0x/ week(s),90.0,7.5
3139,142269.0,No,70.0,NaN,NaN,NaN,NaN,NaN,NaN
3140,142280.0,Yes,21.0,1.0x/ week(s),120.0,2.0,1.0x/ week(s),60.0,1.0
3141,142283.0,Yes,35.0,NaN,NaN,NaN,2.0x/ week(s),60.0,2.0


In [58]:
# Confirm only specified values remain in fields
print(exercise_df['Moderate_LTPA_Hrs_Wk'].value_counts())

# Check value & N/A counts
print('Values: ',exercise_df['Moderate_LTPA_Hrs_Wk'].value_counts().sum())
print('N/A: ',exercise_df['Moderate_LTPA_Hrs_Wk'].isna().sum())
print('Total: ',exercise_df['Moderate_LTPA_Hrs_Wk'].isna().value_counts().sum())

Moderate_LTPA_Hrs_Wk
3.000000     106
1.000000      98
2.000000      95
1.500000      84
4.000000      52
            ... 
60.000000      1
4.416667       1
28.500000      1
4.750000       1
1.250000       1
Name: count, Length: 230, dtype: int64
Values:  1423
N/A:  1720
Total:  3143


#### Dietary

In [59]:
# Load dietary dataset XPT files
dietary_behavior_17_to_pre20 = pd.read_sas('data/P_DBQ.XPT')
dietary_behavior_21_to_23 = pd.read_sas('data/DBQ_L.XPT')

dietary_interview_17_to_pre20 = pd.read_sas('data/P_DR1TOT.XPT')
dietary_interview_21_to_23 = pd.read_sas('data/DR1TOT_L.XPT')

weight_history_17_to_pre20 = pd.read_sas('data/P_WHQ.XPT')
weight_history_21_to_23 = pd.read_sas('data/WHQ_L.XPT')

dietary_supp_17_to_pre20 = pd.read_sas('data/P_DSQTOT.XPT')
dietary_supp_21_to_23 = pd.read_sas('data/DSQTOT_L.XPT')

In [60]:
# Keep all columns
dietary_behavior = pd.concat([dietary_behavior_17_to_pre20, dietary_behavior_21_to_23], axis=0, ignore_index=True)
dietary_interview = pd.concat([dietary_interview_17_to_pre20, dietary_interview_21_to_23], axis=0, ignore_index=True)
weight_history = pd.concat([weight_history_17_to_pre20, weight_history_21_to_23], axis=0, ignore_index=True)
dietary_supp = pd.concat([dietary_supp_17_to_pre20, dietary_supp_21_to_23], axis=0, ignore_index=True)

In [61]:
# Merge cycle and dietary on SEQN (common ID)
dietary_df = pd.merge(cycle_df, dietary_behavior, on='SEQN', how='left')
dietary_df = pd.merge(dietary_df, dietary_interview, on='SEQN', how='left')
dietary_df = pd.merge(dietary_df, weight_history, on='SEQN', how='left')
dietary_df = pd.merge(dietary_df, dietary_supp, on='SEQN', how='left')

In [62]:
# Rename variables of interest
dietary_df.rename(columns={
    'DBQ700': 'Healthiness',
    'DBD895': 'Meals_Not_From_Home_Wk',
    'DBD900': 'Meals_From_Fast_Food_Wk',
    'DRQSPREP': 'Salt_Used',
    'DRQSDIET': 'On_Diet',
    'DR1_320Z': 'Water_Day',
    'WHD080A': 'Ate_Less',
    'WHD080B': 'Lower_Cals',
    'WHD080C': 'Less_Fat',
    'WHD080E': 'Skipped_Meals',
    'WHD080F': 'Diet_Foods',
    'WHD080O': 'Fewer_Carbs',
    'WHD080S': 'Less_Sugar',
    'WHD080T': 'Less_Junk',
    'DSD010': 'Dietary_Supp'
}, inplace=True)

In [63]:
# Map categorical values
dietary_df['Healthiness'] = dietary_df['Healthiness'].map({
    1: 'Excellent',
    2: 'Very good',
    3: 'Good',
    4: 'Fair',
    5: 'Poor'
})

dietary_df['Salt_Used'] = dietary_df['Salt_Used'].map({
    1: 'Never',
    2: 'Rarely',
    3: 'Occasionally',
    4: 'Very often'
})

dietary_df['On_Diet'] = dietary_df['On_Diet'].map({
    1: 'Yes',
    2: 'No'
})

dietary_df['Dietary_Supp'] = dietary_df['Dietary_Supp'].map({
    1: 'Yes',
    2: 'No'
})

#Using replace over map since range values need to be retained
dietary_df['Meals_Not_From_Home_Wk'] = dietary_df['Meals_Not_From_Home_Wk'].replace(5.397605346934028e-79, 'None').replace(5555, '>21').replace([7777, 9999], np.nan)
dietary_df['Meals_From_Fast_Food_Wk'] = dietary_df['Meals_From_Fast_Food_Wk'].replace(5.397605346934028e-79, 'None').replace(5555, '>21').replace([7777, 9999], np.nan)
dietary_df['Ate_Less'] = dietary_df['Ate_Less'].replace(10, 'Ate less')
dietary_df['Lower_Cals'] = dietary_df['Lower_Cals'].replace(11, 'Lower cals')
dietary_df['Less_Fat'] = dietary_df['Less_Fat'].replace(12, 'Less fat')
dietary_df['Skipped_Meals'] = dietary_df['Skipped_Meals'].replace(14, 'Skipped meals')
dietary_df['Diet_Foods'] = dietary_df['Diet_Foods'].replace(15, 'Diet foods')
dietary_df['Fewer_Carbs'] = dietary_df['Fewer_Carbs'].replace(41, 'Fewer carbs')
dietary_df['Less_Sugar'] = dietary_df['Less_Sugar'].replace(45, 'Less sugar')
dietary_df['Less_Junk'] = dietary_df['Less_Junk'].replace(46, 'Less junk')

In [64]:
# No additional rows need to be dropped

In [65]:
# No additional calculation columns needed

In [66]:
#Retain variables of interest only
dietary_df = dietary_df[['SEQN','Regular_Periods', 'Healthiness', 'Meals_Not_From_Home_Wk', 'Meals_From_Fast_Food_Wk', 'Salt_Used', 'On_Diet', 'Water_Day', 'Ate_Less', 'Lower_Cals', 'Less_Fat', 'Skipped_Meals', 'Diet_Foods', 'Fewer_Carbs', 'Less_Sugar', 'Less_Junk', 'Dietary_Supp']]


In [67]:
dietary_df

,SEQN,Regular_Periods,Healthiness,Meals_Not_From_Home_Wk,Meals_From_Fast_Food_Wk,Salt_Used,On_Diet,Water_Day,Ate_Less,Lower_Cals,Less_Fat,Skipped_Meals,Diet_Foods,Fewer_Carbs,Less_Sugar,Less_Junk,Dietary_Supp
0,109266.0,Yes,Good,7.0,None,Very often,Yes,2268.9,NaN,Lower cals,NaN,NaN,NaN,NaN,NaN,Less junk,No
1,109284.0,Yes,Good,1.0,None,Very often,No,1965.0,NaN,Lower cals,NaN,NaN,NaN,Fewer carbs,NaN,Less junk,Yes
2,109291.0,Yes,Very good,1.0,1.0,Rarely,Yes,780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
3,109297.0,Yes,Fair,7.0,3.0,Occasionally,No,870.0,NaN,NaN,Less fat,Skipped meals,Diet foods,NaN,NaN,NaN,No
4,109312.0,Yes,Very good,None,NaN,Rarely,No,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,142263.0,Yes,NaN,NaN,NaN,Very often,No,811.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
3139,142269.0,No,NaN,NaN,NaN,Very often,No,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3140,142280.0,Yes,NaN,NaN,NaN,Very often,Yes,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3141,142283.0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
#Confirm only specified values remain in fields
print(dietary_df['Healthiness'].value_counts())

#Check value & N/A counts
print('Values: ',dietary_df['Healthiness'].value_counts().sum())
print('N/A: ',dietary_df['Healthiness'].isna().sum())
print('Total: ',dietary_df['Healthiness'].isna().value_counts().sum())

Healthiness
Good         790
Fair         586
Very good    340
Poor         152
Excellent    107
Name: count, dtype: int64
Values:  1975
N/A:  1168
Total:  3143


#### Sleep

In [69]:
# Load smoking dataset XPT files
sleep_17_to_pre20 = pd.read_sas('data/P_SLQ.XPT')
sleep_21_to_23 = pd.read_sas('data/SLQ_L.XPT')

# See what’s inside
print(sleep_17_to_pre20.shape)
print(sleep_21_to_23.shape)

(10195, 11)
(8501, 7)


In [70]:
# Keep all columns
sleep_df = pd.concat([sleep_17_to_pre20, sleep_21_to_23], axis=0, ignore_index=True)

In [71]:
# Merge cycle and sleep on SEQN (common ID)
sleep_df = pd.merge(cycle_df, sleep_df, on='SEQN', how='left')

In [72]:
# Rename variables of interest
sleep_df.rename(columns={
    'SLQ300': 'Work_Sleep_Time', 
    'SLQ310': 'Work_Wake_Time', 
    'SLD012': 'Work_Sleep_Hrs', 
    'SLQ320': 'Nonwork_Sleep_Time', 
    'SLQ330': 'Nonwork_Wake_Time',
    'SLD013': 'Nonwork_Sleep_Hrs',
    'SLQ050': 'Trouble_Sleeping',
    'SLQ120': 'Daytime_Sleepy'
}, inplace=True)

In [73]:
# Map categorical values
sleep_df['Trouble_Sleeping'] = sleep_df['Trouble_Sleeping'].map({
    1: 'Yes', 
    2: 'No'})
sleep_df['Daytime_Sleepy'] = sleep_df['Daytime_Sleepy'].map({
    5.397605346934028e-79: 'Never',
    1: 'Rarely', 
    2: 'Sometimes',
    3: 'Often',
    4: 'Almost always'})

# Using replace over map since range values need to be retained
sleep_df['Work_Sleep_Time'] = sleep_df['Work_Sleep_Time'].replace([77777, 99999], np.nan)
sleep_df['Work_Wake_Time'] = sleep_df['Work_Wake_Time'].replace([77777, 99999], np.nan)
sleep_df['Work_Sleep_Hrs'] = sleep_df['Work_Sleep_Hrs'].replace(2, '< 3').replace(14, '>= 14')
sleep_df['Nonwork_Sleep_Time'] = sleep_df['Work_Sleep_Time'].replace([77777, 99999], np.nan)
sleep_df['Nonwork_Wake_Time'] = sleep_df['Work_Sleep_Time'].replace([77777, 99999], np.nan)
sleep_df['Nonwork_Sleep_Hrs'] = sleep_df['Nonwork_Sleep_Hrs'].replace(2, '< 3').replace(14, '>= 14')

In [74]:
# No additional rows need to be dropped

In [75]:
#Convert time into hour #'s
sleep_df['Work_Sleep_Time_Decoded'] = sleep_df['Work_Sleep_Time'].str.decode('utf-8')
sleep_df['Work_Sleep_Time_Decoded'] = pd.to_datetime(sleep_df['Work_Sleep_Time_Decoded'], format='%H:%M', errors='coerce').dt.hour

sleep_df['Work_Wake_Time_Decoded'] = sleep_df['Work_Wake_Time'].str.decode('utf-8')
sleep_df['Work_Wake_Time_Decoded'] = pd.to_datetime(sleep_df['Work_Wake_Time_Decoded'], format='%H:%M', errors='coerce').dt.hour

sleep_df['Nonwork_Sleep_Time_Decoded'] = sleep_df['Nonwork_Sleep_Time'].str.decode('utf-8')
sleep_df['Nonwork_Sleep_Time_Decoded'] = pd.to_datetime(sleep_df['Nonwork_Sleep_Time_Decoded'], format='%H:%M', errors='coerce').dt.hour

sleep_df['Nonwork_Wake_Time_Decoded'] = sleep_df['Nonwork_Wake_Time'].str.decode('utf-8')
sleep_df['Nonwork_Wake_Time_Decoded'] = pd.to_datetime(sleep_df['Nonwork_Wake_Time_Decoded'], format='%H:%M', errors='coerce').dt.hour


# Split into early, average, or late sleep/wake time
sleep_df['Work_Sleep_Time_Category'] = pd.cut(
    sleep_df['Work_Sleep_Time_Decoded'], 
    bins=[0, 22, 24, 25], 
    labels=['Early (before 10pm)', 'Normal (10pm-12am)', 'Late (after 12am)'],
    include_lowest=True,
    right=False
)

sleep_df['Work_Wake_Time_Category'] = pd.cut(
    sleep_df['Work_Wake_Time_Decoded'], 
    bins=[0, 6, 8, 24],
    labels=['Early (before 6am)', 'Normal (6am-8am)', 'Late (after 8am)'],
    include_lowest=True,
    right=False
)

sleep_df['Nonwork_Sleep_Time_Category'] = pd.cut(
    sleep_df['Nonwork_Sleep_Time_Decoded'], 
    bins=[0, 22, 24, 25], 
    labels=['Early (before 10pm)', 'Normal (10pm-12am)', 'Late (after 12am)'],
    include_lowest=True,
    right=False
)

sleep_df['Nonwork_Wake_Time_Category'] = pd.cut(
    sleep_df['Nonwork_Wake_Time_Decoded'], 
    bins=[0, 6, 8, 24],
    labels=['Early (before 6am)', 'Normal (6am-8am)', 'Late (after 8am)'],
    include_lowest=True,
    right=False
)

#Convert # into numeric
sleep_df['Work_Sleep_Hrs'] = pd.to_numeric(sleep_df['Work_Sleep_Hrs'], errors='coerce')
sleep_df['Nonwork_Sleep_Hrs'] = pd.to_numeric(sleep_df['Nonwork_Sleep_Hrs'], errors='coerce')

# Split into short, average, or long time spent sleeping
sleep_df['Work_Sleep_Hrs_Category'] = pd.cut(
    sleep_df['Work_Sleep_Hrs'], 
    bins=[0, 7, 9, 24],
    labels=['Short (<7hrs)', 'Normal (7-9hrs)', 'Long (>9hrs)'],
    include_lowest=True,
    right=False
)

sleep_df['Nonwork_Sleep_Hrs_Category'] = pd.cut(
    sleep_df['Nonwork_Sleep_Hrs'], 
    bins=[0, 7, 9, 24],
    labels=['Short (<7hrs)', 'Normal (7-9hrs)', 'Long (>9hrs)'],
    include_lowest=True,
    right=False
)

In [76]:
# Retain variables of interest only
sleep_df = sleep_df[['SEQN', 'Regular_Periods', 'Work_Sleep_Time_Category', 'Work_Wake_Time_Category', 'Work_Sleep_Hrs', 'Work_Sleep_Hrs_Category', 'Nonwork_Sleep_Time_Category', 'Nonwork_Wake_Time_Category', 'Nonwork_Sleep_Hrs', 'Nonwork_Sleep_Hrs_Category', 'Trouble_Sleeping', 'Daytime_Sleepy']]


In [77]:
sleep_df

,SEQN,Regular_Periods,Work_Sleep_Time_Category,Work_Wake_Time_Category,Work_Sleep_Hrs,Work_Sleep_Hrs_Category,Nonwork_Sleep_Time_Category,Nonwork_Wake_Time_Category,Nonwork_Sleep_Hrs,Nonwork_Sleep_Hrs_Category,Trouble_Sleeping,Daytime_Sleepy
0,109266.0,Yes,Normal (10pm-12am),Early (before 6am),7.5,Normal (7-9hrs),Normal (10pm-12am),Late (after 8am),8.0,Normal (7-9hrs),No,Never
1,109284.0,Yes,Normal (10pm-12am),Normal (6am-8am),8.0,Normal (7-9hrs),Normal (10pm-12am),Late (after 8am),10.0,Long (>9hrs),No,Never
2,109291.0,Yes,Early (before 10pm),Late (after 8am),11.0,Long (>9hrs),Early (before 10pm),Early (before 6am),10.0,Long (>9hrs),Yes,Never
3,109297.0,Yes,Early (before 10pm),Late (after 8am),8.0,Normal (7-9hrs),Early (before 10pm),Early (before 6am),8.0,Normal (7-9hrs),No,Never
4,109312.0,Yes,Normal (10pm-12am),Normal (6am-8am),8.5,Normal (7-9hrs),Normal (10pm-12am),Late (after 8am),7.5,Normal (7-9hrs),No,Rarely
...,...,...,...,...,...,...,...,...,...,...,...,...
3138,142263.0,Yes,Normal (10pm-12am),Normal (6am-8am),9.0,Long (>9hrs),Normal (10pm-12am),Late (after 8am),10.0,Long (>9hrs),NaN,NaN
3139,142269.0,No,Early (before 10pm),Late (after 8am),5.0,Short (<7hrs),Early (before 10pm),Late (after 8am),5.0,Short (<7hrs),NaN,NaN
3140,142280.0,Yes,Early (before 10pm),Late (after 8am),9.0,Long (>9hrs),Early (before 10pm),Early (before 6am),9.0,Long (>9hrs),NaN,NaN
3141,142283.0,Yes,Early (before 10pm),Late (after 8am),8.0,Normal (7-9hrs),Early (before 10pm),Early (before 6am),8.0,Normal (7-9hrs),NaN,NaN


In [78]:
# Confirm only specified values remain in fields
print(sleep_df['Daytime_Sleepy'].value_counts())

# Check value & N/A counts
print('Values: ',sleep_df['Daytime_Sleepy'].value_counts().sum())
print('N/A: ',sleep_df['Daytime_Sleepy'].isna().sum())
print('Total: ',sleep_df['Daytime_Sleepy'].isna().value_counts().sum())

Daytime_Sleepy
Sometimes        673
Often            429
Rarely           424
Never            249
Almost always    200
Name: count, dtype: int64
Values:  1975
N/A:  1168
Total:  3143


#### Dataframes:

In [80]:
bmi_df

,SEQN,BMI,Waist_circum_cm,Hip_Circum_cm,Attempted_Weight_Loss,Height_in,Weight_lbs,Weight_Change_lbs
0,109266.0,37.8,117.9,126.1,Yes,64.0,210.0,-10.0
1,109284.0,39.1,103.1,125.5,Yes,60.0,178.0,-8.0
2,109291.0,31.3,NaN,NaN,No,64.0,148.0,-8.0
3,109297.0,23.2,73.2,92.5,Yes,60.0,120.0,-5.0
4,109312.0,23.9,95.7,96.3,No,66.0,139.0,0.0
...,...,...,...,...,...,...,...,...
3138,142263.0,22.6,80.4,95.0,No,65.0,135.0,0.0
3139,142269.0,NaN,149.3,147.2,No,64.0,350.0,0.0
3140,142280.0,38.4,124.1,130.0,Yes,69.0,250.0,20.0
3141,142283.0,45.8,137.2,145.7,Yes,67.0,275.0,5.0


In [81]:
smoking_df

,SEQN,Regular_Periods,Smoked_100,Active_Smoker,Lifetime_Cig_Count,Household_Smoker_Count,Last_Cig
0,109266.0,Yes,No,NaN,NaN,0,NaN
1,109284.0,Yes,No,NaN,NaN,0,NaN
2,109291.0,Yes,No,NaN,NaN,0,NaN
3,109297.0,Yes,No,NaN,NaN,0,Yesterday
4,109312.0,Yes,No,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...
3138,142263.0,Yes,No,NaN,NaN,0,NaN
3139,142269.0,No,No,NaN,NaN,0,NaN
3140,142280.0,Yes,No,NaN,NaN,0,NaN
3141,142283.0,Yes,No,NaN,NaN,0,NaN


In [84]:
exercise_df

,SEQN,Regular_Periods,Sedentary_Hrs_Wk,Moderate_LTPA_Freq,Moderate_LTPA_Mins,Moderate_LTPA_Hrs_Wk,Vigorous_LTPA_Freq,Vigorous_LTPA_Mins,Vigorous_LTPA_Hrs_Wk
0,109266.0,Yes,56.0,NaN,NaN,NaN,NaN,NaN,NaN
1,109284.0,Yes,42.0,NaN,NaN,NaN,NaN,NaN,NaN
2,109291.0,Yes,63.0,NaN,NaN,NaN,NaN,NaN,NaN
3,109297.0,Yes,70.0,NaN,NaN,NaN,NaN,NaN,NaN
4,109312.0,Yes,21.0,12.0x/ week,480.0,52.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3138,142263.0,Yes,56.0,5.0x/ week(s),90.0,7.5,5.0x/ week(s),90.0,7.5
3139,142269.0,No,70.0,NaN,NaN,NaN,NaN,NaN,NaN
3140,142280.0,Yes,21.0,1.0x/ week(s),120.0,2.0,1.0x/ week(s),60.0,1.0
3141,142283.0,Yes,35.0,NaN,NaN,NaN,2.0x/ week(s),60.0,2.0


In [85]:
demo_df

,SEQN,Regular_Periods,Race,Age,Education,Family_Income_to_Poverty_Ratio
1,109266.0,Yes,Non-Hispanic Asian,29.0,College graduate or above,5.00
4,109284.0,Yes,Mexican American,44.0,9-11th grade,NaN
5,109291.0,Yes,Non-Hispanic Asian,42.0,College graduate or above,NaN
7,109297.0,Yes,Non-Hispanic Asian,30.0,Some college/AA,NaN
8,109312.0,Yes,Non-Hispanic White,48.0,College graduate or above,5.00
...,...,...,...,...,...,...
4454,142263.0,Yes,Non-Hispanic White,44.0,College graduate or above,NaN
4455,142269.0,No,Non-Hispanic Black,32.0,Some college/AA,0.74
4456,142280.0,Yes,Other Race - Including Multi-Racial,23.0,Some college/AA,1.40
4457,142283.0,Yes,Other Race - Including Multi-Racial,29.0,High school/GED,1.04


In [86]:
dietary_df

,SEQN,Regular_Periods,Healthiness,Meals_Not_From_Home_Wk,Meals_From_Fast_Food_Wk,Salt_Used,On_Diet,Water_Day,Ate_Less,Lower_Cals,Less_Fat,Skipped_Meals,Diet_Foods,Fewer_Carbs,Less_Sugar,Less_Junk,Dietary_Supp
0,109266.0,Yes,Good,7.0,None,Very often,Yes,2268.9,NaN,Lower cals,NaN,NaN,NaN,NaN,NaN,Less junk,No
1,109284.0,Yes,Good,1.0,None,Very often,No,1965.0,NaN,Lower cals,NaN,NaN,NaN,Fewer carbs,NaN,Less junk,Yes
2,109291.0,Yes,Very good,1.0,1.0,Rarely,Yes,780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
3,109297.0,Yes,Fair,7.0,3.0,Occasionally,No,870.0,NaN,NaN,Less fat,Skipped meals,Diet foods,NaN,NaN,NaN,No
4,109312.0,Yes,Very good,None,NaN,Rarely,No,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,142263.0,Yes,NaN,NaN,NaN,Very often,No,811.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
3139,142269.0,No,NaN,NaN,NaN,Very often,No,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3140,142280.0,Yes,NaN,NaN,NaN,Very often,Yes,1920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3141,142283.0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
sleep_df

,SEQN,Regular_Periods,Work_Sleep_Time_Category,Work_Wake_Time_Category,Work_Sleep_Hrs,Work_Sleep_Hrs_Category,Nonwork_Sleep_Time_Category,Nonwork_Wake_Time_Category,Nonwork_Sleep_Hrs,Nonwork_Sleep_Hrs_Category,Trouble_Sleeping,Daytime_Sleepy
0,109266.0,Yes,Normal (10pm-12am),Early (before 6am),7.5,Normal (7-9hrs),Normal (10pm-12am),Late (after 8am),8.0,Normal (7-9hrs),No,Never
1,109284.0,Yes,Normal (10pm-12am),Normal (6am-8am),8.0,Normal (7-9hrs),Normal (10pm-12am),Late (after 8am),10.0,Long (>9hrs),No,Never
2,109291.0,Yes,Early (before 10pm),Late (after 8am),11.0,Long (>9hrs),Early (before 10pm),Early (before 6am),10.0,Long (>9hrs),Yes,Never
3,109297.0,Yes,Early (before 10pm),Late (after 8am),8.0,Normal (7-9hrs),Early (before 10pm),Early (before 6am),8.0,Normal (7-9hrs),No,Never
4,109312.0,Yes,Normal (10pm-12am),Normal (6am-8am),8.5,Normal (7-9hrs),Normal (10pm-12am),Late (after 8am),7.5,Normal (7-9hrs),No,Rarely
...,...,...,...,...,...,...,...,...,...,...,...,...
3138,142263.0,Yes,Normal (10pm-12am),Normal (6am-8am),9.0,Long (>9hrs),Normal (10pm-12am),Late (after 8am),10.0,Long (>9hrs),NaN,NaN
3139,142269.0,No,Early (before 10pm),Late (after 8am),5.0,Short (<7hrs),Early (before 10pm),Late (after 8am),5.0,Short (<7hrs),NaN,NaN
3140,142280.0,Yes,Early (before 10pm),Late (after 8am),9.0,Long (>9hrs),Early (before 10pm),Early (before 6am),9.0,Long (>9hrs),NaN,NaN
3141,142283.0,Yes,Early (before 10pm),Late (after 8am),8.0,Normal (7-9hrs),Early (before 10pm),Early (before 6am),8.0,Normal (7-9hrs),NaN,NaN
